# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../Output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,14.90,92,100,3.13,NZ,1682140587
1,1,xining,36.6167,101.7667,8.34,49,100,3.95,CN,1682140587
2,2,bethel,41.3712,-73.4140,10.81,85,100,3.60,US,1682140588
3,3,port-aux-francais,-49.3500,70.2167,5.12,74,85,10.92,TF,1682140588
4,4,tacuarembo,-31.7333,-55.9833,13.84,56,100,2.46,UY,1682140501


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
# %%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_cities_df = city_data_df.loc[(city_data_df["Max Temperature"]>=22) & 
                                     (city_data_df["Max Temperature"]<=32) &
                                     (city_data_df["Humidity"]<50) &
                                     (city_data_df["Wind Speed"]<4),:]

# Drop any rows with null values
perfect_cities_df = perfect_cities_df.dropna()

# Display sample data
perfect_cities_df

,City_ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,mopti,14.6667,-4.0000,27.66,14,12,3.48,ML,1682140613
145,145,birjand,32.8663,59.2211,26.26,10,0,2.06,IR,1682140631
213,213,nyala,12.0500,24.8833,27.06,12,29,3.17,SD,1682140651
228,228,dedougou,12.4667,-3.4667,28.41,19,57,1.11,BF,1682140550
250,250,goz beida,13.9500,20.5500,27.73,15,15,2.75,TD,1682140662
275,275,adrar,20.5022,-10.0711,23.88,18,88,3.69,MR,1682140672
288,288,naracoorte,-36.9579,140.7383,22.10,46,1,3.24,AU,1682140675
380,380,ghat,24.9647,10.1728,22.76,17,50,1.82,LY,1682140700
384,384,atar,20.5169,-13.0499,25.49,18,83,1.96,MR,1682140702
410,410,margaret river,-33.9500,115.0667,23.90,37,0,2.88,AU,1682140710


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_cities_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
85,85,mopti,14.6667,-4.0000,27.66,14,12,3.48,ML,1682140613,
145,145,birjand,32.8663,59.2211,26.26,10,0,2.06,IR,1682140631,
213,213,nyala,12.0500,24.8833,27.06,12,29,3.17,SD,1682140651,
228,228,dedougou,12.4667,-3.4667,28.41,19,57,1.11,BF,1682140550,
250,250,goz beida,13.9500,20.5500,27.73,15,15,2.75,TD,1682140662,
275,275,adrar,20.5022,-10.0711,23.88,18,88,3.69,MR,1682140672,
288,288,naracoorte,-36.9579,140.7383,22.10,46,1,3.24,AU,1682140675,
380,380,ghat,24.9647,10.1728,22.76,17,50,1.82,LY,1682140700,
384,384,atar,20.5169,-13.0499,25.49,18,83,1.96,MR,1682140702,
410,410,margaret river,-33.9500,115.0667,23.90,37,0,2.88,AU,1682140710,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [53]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key,
          "categories":"accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mopti - nearest hotel: No hotel found
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
nyala - nearest hotel: سوني سفاري
dedougou - nearest hotel: No hotel found
goz beida - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
naracoorte - nearest hotel: Kincraig Hotel /Smith & Jones Cafe
ghat - nearest hotel: فندق تاسيلي
atar - nearest hotel: فندق سكليل
margaret river - nearest hotel: Margaret River Hotel
long beach - nearest hotel: Westin
turpan - nearest hotel: Oriental
xichang - nearest hotel: No hotel found
gereida - nearest hotel: No hotel found
pennagaram - nearest hotel: No hotel found
berber - nearest hotel: No hotel found


,City_ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
85,85,mopti,14.6667,-4.0000,27.66,14,12,3.48,ML,1682140613,No hotel found
145,145,birjand,32.8663,59.2211,26.26,10,0,2.06,IR,1682140631,مهمانسرای جهانگردی بیرجند
213,213,nyala,12.0500,24.8833,27.06,12,29,3.17,SD,1682140651,سوني سفاري
228,228,dedougou,12.4667,-3.4667,28.41,19,57,1.11,BF,1682140550,No hotel found
250,250,goz beida,13.9500,20.5500,27.73,15,15,2.75,TD,1682140662,No hotel found
275,275,adrar,20.5022,-10.0711,23.88,18,88,3.69,MR,1682140672,No hotel found
288,288,naracoorte,-36.9579,140.7383,22.10,46,1,3.24,AU,1682140675,Kincraig Hotel /Smith & Jones Cafe
380,380,ghat,24.9647,10.1728,22.76,17,50,1.82,LY,1682140700,فندق تاسيلي
384,384,atar,20.5169,-13.0499,25.49,18,83,1.96,MR,1682140702,فندق سكليل
410,410,margaret river,-33.9500,115.0667,23.90,37,0,2.88,AU,1682140710,Margaret River Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
# %%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name","Country"],
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)